In [ ]:
%reload keras_gym.base.mixins keras_gym.base.errors keras_gym.base
%reload keras_gym.caching.experience_replay keras_gym.caching
%reload keras_gym.utils.misc keras_gym.utils.tensor keras_gym.utils
%reload keras_gym.function_approximators.base keras_gym.function_approximators.generic
%reload keras_gym.function_approximators.actor_critic keras_gym.function_approximators keras_gym
import keras_gym as km
import tensorflow as tf
keras = tf.keras
K = keras.backend
K.clear_session()

env = gym.make('Pendulum-v0')
env = km.wrappers.TrainMonitor(env)
km.enable_logging()


class MLP(km.FunctionApproximator):
    def body(self, X, variable_scope):
        X = keras.layers.Dense(units=6, name=f'{variable_scope}/dense1')(X)
        X = keras.layers.Dense(units=6, name=f'{variable_scope}/dense2')(X)
        return X


mlp = MLP(env, lr=1e-15)
pi = km.BetaPolicy(mlp, update_strategy='ppo')
v = km.V(mlp, gamma=0.9)
actor_critic = km.ActorCritic(pi, v)
actor_critic.train_model.summary()

buffer = km.caching.ExperienceReplayBuffer.from_value_function(
    v, capacity=256, batch_size=64)


while env.T < 1000000:
    s = env.reset()

    for t in range(env.spec.max_episode_steps):
        a = pi(s, use_target_model=True)  # target_model == pi_old
        s_next, r, done, info = env.step(a)

        buffer.add(s, a, r, done, env.ep)

        if len(buffer) >= buffer.capacity:
            raise Exception()
            # use 4 epochs per round
            num_batches = int(4 * buffer.capacity / buffer.batch_size)
            for _ in range(num_batches):
                actor_critic.batch_update(*buffer.sample())
            buffer.clear()

            # soft update (tau=1 would be a hard update)
            actor_critic.sync_target_model(tau=0.1)

        if done:
            break

        s = s_next

env.close()

In [ ]:
print(v(s))

batch = list(buffer.sample())
batch[1][:,:,1] = 10.
batch[5][:,:,1] = 10.

actor_critic.batch_update(*batch)
# pi.dist_params(s)
v(s)

# actor_critic.batch_eval(batch[0])

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
K.get_session().run(tf.is_finite(K.constant(0)))